# After getting files

## Insert into database

In [1]:
from pathlib import Path
try:
    dataset
except:
    print('getting dataset location')
    dataset = Path('dataset').absolute()


getting dataset location


In [2]:
import os
os.chdir(dataset)

js_dir = dataset / 'javascript'


In [6]:
import xml.etree.ElementTree as ET

In [43]:
import re

html_tags = '|'.join([
    # Basic HTML Document Structure
    "!DOCTYPE", "html", "head", "title", "body",

    # Text Formatting
    "h1", "h2", "h3", "h4", "h5", "h6", "p", "br", "hr", "abbr", "acronym", "address", "b", "bdi", "bdo",
    "blockquote", "cite", "code", "del", "dfn", "em", "i", "ins", "kbd", "mark", "meter", "pre", "progress",
    "q", "rp", "rt", "ruby", "s", "samp", "small", "strike", "strong", "sub", "sup", "template", "time",
    "tt", "u", "var", "wbr",

    # Links and Resources
    "a", "link", "nav", "ul", "ol", "li", "dir", "dl", "dt", "dd", "figure", "figcaption", "main",

    # Images and Multimedia
    "img", "map", "area", "canvas", "figcaption", "figure", "picture", "svg", "audio", "source", "track", "video",

    # Tables
    "table", "caption", "th", "tr", "td", "thead", "tbody", "tfoot", "col", "colgroup",

    # Forms
    "form", "input", "textarea", "button", "select", "optgroup", "option", "label", "fieldset", "legend",
    "datalist", "output",

    # Frames
    "frame", "frameset", "noframes", "iframe",

    # Styles and Semantics
    "style", "div", "span", "header", "footer", "section", "article", "aside",

    # Meta Info
    "meta", "base", "basefont",

    # Programming
    "script", "noscript", "embed", "object", "param"
])

def simple_parse_xml(content, nested, allow_jsx = True):
    pattern = r'<([^\s]*?)>'
    if not allow_jsx:
        pattern = rf'<((?!({html_tags})>)[^\sA-Z][^\s]*?)>'
    open = re.search(pattern, content)
    if not open:
        return [content]
    [open_start, open_end] = open.span()
    tag = open.group(1)
    before = content[:open_start]
    inner_and_after = content[open_end:]
    close = re.search(f'</{re.escape(tag)}>', inner_and_after)
    if not close:
        return [content]
    [close_start, close_end] = close.span()
    inner = inner_and_after[:close_start]
    after = inner_and_after[close_end:]
    inner = simple_parse_xml(inner, nested, False) if nested else inner
    return [[before, tag, inner], *simple_parse_xml(after, nested, allow_jsx)]



In [37]:
[p[1] for p in parsed]

['function1',
 'buildScales',
 'prepareProject',
 'initializeProperties',
 'defineTranslations',
 '}']

In [64]:
import json
omitted = []
output = []
for text_file in js_dir.glob('thread2-*.txt'):
    with open(text_file, 'r') as file:
        file_contents = file.read()
    parsed = simple_parse_xml(file_contents, nested=False)
    output.append({
        'file': str(text_file),
        'split': [p[0] for p in parsed[:-1]] + [parsed[-1]],
        'len': len(parsed),
        'ok': len(parsed) == 11
    })
    if len(parsed) != 11:
        omitted.append(text_file)
        continue
    with open(str.replace(str(text_file), '.txt', '.json'), 'w') as file:
        file.write(json.dumps([p[1:] for p in parsed]))

with open('parse1.log', 'w') as file:
    file.write(json.dumps(output))
for out in output:
    print(out['file'])
    print(out['split'])

/root/py/CodeBERT/CodeReviewer/dataset/javascript/thread2-35.txt
['', '\n\n', '\n\n', '\n\n', '\n\n', '\n\n', '\n\n', '\n\n', '\n\n', '\n\n', '']
/root/py/CodeBERT/CodeReviewer/dataset/javascript/thread2-100.txt
['```javascript\n', '\n```\n\n```javascript\n', '\n```\n\n```javascript\n', '\n```\n\n```javascript\n', '\n```\n\n```javascript\n', '\n```\n\n```javascript\n', '\n```\n\n```javascript\n', '\n```\n\n```javascript\n', '\n```\n\n```javascript\n', '\n```\n\n```javascript\n', "\n```\n\nIn all these annotations, I've only tagged lines or blocks of code with distinct, identifiable responsibilities, following the balanced SRP approach, avoiding over-segmenting and under-segmenting. Each tagged responsibility represents a specific purpose within the function."]
/root/py/CodeBERT/CodeReviewer/dataset/javascript/thread2-23.txt
['', '\n\n', '\n\n', '\n\n', '\n\n', '\n\n', '\n\n', '\n\n', '\n\n', '\n\n', '']
/root/py/CodeBERT/CodeReviewer/dataset/javascript/thread2-217.txt
['Here are the an

In [67]:
len(omitted)

25

In [66]:
len(output)

211